In [ ]:
import pandas as pd

g4a_main_eval = pd.read_csv("evaluation/judge_outputs/LLM_judge_g4a_few_shot_new_main_train_all.csv")
g4a_main_eval.rename(columns={"variable": "cl"}, inplace=True)
g4a_main_eval.head()

In [ ]:
filter_ids_content_falsch = set(g4a_main_eval[g4a_main_eval.LLM_judge_content_preservation == "falsch"].id.values)
len(filter_ids_content_falsch)

In [ ]:
filter_ids_zu_einfach = set(g4a_main_eval[(g4a_main_eval.LLM_judge_complexity_level.isin(["zu_einfach"])) & (g4a_main_eval.cl != "cl_1")].id.values)
filter_ids_zu_kompliziert = set(g4a_main_eval[(g4a_main_eval.LLM_judge_complexity_level.isin(["zu_kompliziert"])) & (g4a_main_eval.cl != "cl_5")].id.values)
print("zu einfach", len(g4a_main_eval[(g4a_main_eval.LLM_judge_complexity_level.isin(["zu_einfach"]))]))
print("zu kompliziert", len(g4a_main_eval[(g4a_main_eval.LLM_judge_complexity_level.isin(["zu_kompliziert"]))]))
len(filter_ids_zu_einfach), len(filter_ids_zu_kompliziert)

In [ ]:
filter_ids_information_loss = set(g4a_main_eval[(g4a_main_eval.LLM_judge_information_loss.isin(["haeufig"])) & (g4a_main_eval.cl.isin(["cl_5"]))].id.values)
filter_ids_information_add = set(g4a_main_eval[(g4a_main_eval.LLM_judge_information_addition.isin(["haeufig"])) & (g4a_main_eval.cl != "cl_5")].id.values)
len(filter_ids_information_loss), len(filter_ids_information_add)

In [ ]:
for _, row in g4a_main_eval[(g4a_main_eval.LLM_judge_information_loss.isin(["haeufig"])) & (~g4a_main_eval.cl.isin(["cl_1", "cl_2", "cl_3"]))].sample(20).iterrows():
    print(row.id, "- Orig:", row.text)
    print("Para:", row.paraphrased_text)
    print("CL: ", row.cl)
    print()

In [ ]:
from ast import literal_eval

g4a_main_eval["LLM_judge_type_of_addition"] = g4a_main_eval["LLM_judge_type_of_addition"].apply(literal_eval)
g4a_main_eval_explode = g4a_main_eval.explode("LLM_judge_type_of_addition")
filter_ids_inkorrekte_Informationen = set(g4a_main_eval_explode[(g4a_main_eval_explode.LLM_judge_type_of_addition.isin(["Faktisch_inkorrekte_Informationen"]))].id.values)
filter_ids_Andere = set(g4a_main_eval_explode[(g4a_main_eval_explode.LLM_judge_type_of_addition.isin(["Andere"]))].id.values)
filter_ids_korrekte_Informationen = set(g4a_main_eval_explode[(g4a_main_eval_explode.LLM_judge_type_of_addition.isin(["Faktisch_korrekte_Informationen"])) & (g4a_main_eval_explode.cl != "cl_5")].id.values)
len(filter_ids_inkorrekte_Informationen), len(filter_ids_korrekte_Informationen), len(filter_ids_Andere)

In [ ]:
for _, row in g4a_main_eval_explode[(g4a_main_eval_explode.LLM_judge_type_of_addition.isin(["Faktisch_inkorrekte_Informationen"]))].iterrows():
    print("Orig:", row.text)
    print("Para:", row.paraphrased_text)
    print("CL: ", row.cl)
    print()

In [ ]:
for _, row in g4a_main_eval_explode[(g4a_main_eval_explode.LLM_judge_type_of_addition.isin(["Faktisch_korrekte_Informationen"]))
    #].iterrows():
    & (g4a_main_eval_explode.cl != "cl_5")].iterrows():
    print("Orig:", row.text)
    print("Para:", row.paraphrased_text)
    print("CL: ", row.cl)
    print()

In [ ]:
g4a_main_eval[g4a_main_eval.LLM_judge_complexity_level != "passend"].hist(column="LLM_judge_complexity_level", by="cl")

In [ ]:
all_filter_ids = set()
all_filter_ids.add(2018814)
all_filter_ids.add(753704)
for filter_ids in [
    filter_ids_content_falsch, filter_ids_zu_einfach, filter_ids_zu_kompliziert, filter_ids_inkorrekte_Informationen,
    filter_ids_Andere, filter_ids_korrekte_Informationen,
    filter_ids_information_loss
]:
    all_filter_ids = all_filter_ids.union(filter_ids)
len(all_filter_ids)

In [ ]:
german4all_main = pd.read_csv("../German4All-Corpus/main/train.csv")
print(len(german4all_main))

In [ ]:
german4all_main_filtered = german4all_main[~german4all_main.id.isin(all_filter_ids)]
print(len(german4all_main_filtered))

In [ ]:
german4all_main_filtered.to_csv("../German4All-Corpus/main/train.csv", index=False)